In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,8114
2,Huelva,Confirmados PDIA 14 días,2571
3,Huelva,Tasa PDIA 14 días,"503,3842852471791"
4,Huelva,Confirmados PDIA 7 días,1052
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,1
629,Municipio de Huelva sin especificar,Total Confirmados,103
630,Municipio de Huelva sin especificar,Curados,28


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  8114.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2249.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 476 personas en los últimos 7 días 

Un positivo PCR cada 189 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,8114.0,1052.0,2571.0,510743.0,205.974433,503.384285,217.0
Huelva-Costa,4300.0,582.0,1503.0,288115.0,202.002673,521.666696,128.0
Condado-Campiña,2567.0,363.0,757.0,155057.0,234.107457,488.207562,88.0
Huelva (capital),2249.0,302.0,759.0,143663.0,210.214182,528.319748,77.0
Almonte,407.0,75.0,123.0,24191.0,310.032657,508.453557,36.0
Lepe,665.0,83.0,243.0,27431.0,302.577376,885.859065,17.0
Moguer,214.0,45.0,106.0,22088.0,203.730532,479.898587,12.0
Palma del Condado (La),369.0,55.0,107.0,10761.0,511.104916,994.331382,11.0
Ayamonte,206.0,45.0,81.0,20946.0,214.838155,386.708679,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,12.0,9.0,9.0,548.0,1642.335766,1642.335766,0.0
Villablanca,51.0,11.0,36.0,2848.0,386.235955,1264.044944,1.0
Aracena,278.0,18.0,91.0,8107.0,222.030344,1122.486740,1.0
Palma del Condado (La),369.0,55.0,107.0,10761.0,511.104916,994.331382,11.0
Almendro (El),16.0,1.0,8.0,826.0,121.065375,968.523002,0.0
Villarrasa,86.0,10.0,20.0,2176.0,459.558824,919.117647,0.0
Lepe,665.0,83.0,243.0,27431.0,302.577376,885.859065,17.0
Marines (Los),5.0,1.0,3.0,396.0,252.525253,757.575758,0.0
Lucena del Puerto,46.0,18.0,24.0,3371.0,533.966182,711.954910,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,82.0,1.0,9.0,3338.0,29.958059,269.622528,0.0,0.111111
Almendro (El),16.0,1.0,8.0,826.0,121.065375,968.523002,0.0,0.125000
Manzanilla,44.0,1.0,7.0,2135.0,46.838407,327.868852,0.0,0.142857
Aracena,278.0,18.0,91.0,8107.0,222.030344,1122.486740,1.0,0.197802
Almonaster la Real,32.0,1.0,4.0,1819.0,54.975261,219.901045,0.0,0.250000
Galaroza,7.0,1.0,4.0,1379.0,72.516316,290.065265,0.0,0.250000
Gibraleón,326.0,16.0,62.0,12607.0,126.913619,491.790275,4.0,0.258065
Bonares,32.0,5.0,18.0,6058.0,82.535490,297.127765,2.0,0.277778
Zalamea la Real,41.0,2.0,7.0,3068.0,65.189048,228.161669,0.0,0.285714
